# BQNT Carry Execution - Backtest

Historical backtest of the FX/Metals carry strategy.

**Target: Sharpe Ratio ~0.55**

## 1. Setup

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

# BQL
import bql
bq = bql.Service()

# Add src to path
import sys
sys.path.insert(0, '../src')

print(f"Backtest Analysis - {datetime.now().strftime('%Y-%m-%d %H:%M')}")

In [ ]:
# Import modules
from config.tickers import TRADING_UNIVERSE, get_bloomberg_ticker
from config.parameters import SIGNAL_WEIGHTS, BACKTEST_PARAMS
from data.bql_loader import BQLDataLoader
from data.cache import get_cache
from signals.carry import CarrySignalEngine
from signals.momentum import MomentumSignalEngine
from signals.regime import RegimeSignalEngine
from portfolio.construction import PortfolioConstructor
from backtest.engine import BacktestEngine, BacktestResult

print("Modules loaded!")

In [ ]:
# Initialize components
cache = get_cache()
loader = BQLDataLoader(cache=cache)
carry_engine = CarrySignalEngine()
momentum_engine = MomentumSignalEngine()
regime_engine = RegimeSignalEngine()
portfolio = PortfolioConstructor()
backtest_engine = BacktestEngine()

print("Components initialized!")

## 2. Fetch Historical Data

In [ ]:
# Backtest parameters
LOOKBACK_DAYS = 504  # 2 years

print(f"Fetching {LOOKBACK_DAYS} days of historical data...")

In [ ]:
# Fetch all data
print("Fetching FX spot prices...")
fx_data = loader.fetch_fx_data(days=LOOKBACK_DAYS)
spot_prices = fx_data['spot']
forward_prices = fx_data['forward']

print("Fetching metals prices...")
metals_prices = loader.fetch_metals_data(days=LOOKBACK_DAYS)

print("Fetching volatility data...")
vol_data = loader.fetch_volatility_data(days=LOOKBACK_DAYS)

print(f"\nData loaded:")
print(f"  FX Spot: {spot_prices.shape}")
print(f"  FX Forward: {forward_prices.shape}")
print(f"  Metals: {metals_prices.shape}")
print(f"  Volatility: {vol_data.shape}")

In [ ]:
# Combine prices
all_prices = pd.concat([
    spot_prices,
    metals_prices[['XAUUSD', 'XAGUSD']] if 'XAUUSD' in metals_prices.columns else pd.DataFrame()
], axis=1)

# Forward fill missing data
all_prices = all_prices.fillna(method='ffill').dropna()

print(f"\nCombined prices: {all_prices.shape}")
print(f"Date range: {all_prices.index[0]} to {all_prices.index[-1]}")

## 3. Generate Historical Signals

In [ ]:
# Calculate returns and volatility
returns = all_prices.pct_change().dropna()
volatility = returns.rolling(21).std() * np.sqrt(252)

print(f"Returns shape: {returns.shape}")

In [ ]:
# Generate momentum signals
print("Generating momentum signals...")
momentum_signals = momentum_engine.calculate_signals(all_prices)
print(f"  Shape: {momentum_signals.shape}")

In [ ]:
# Generate carry signals
print("Generating carry signals...")
carry_signals = carry_engine.calculate_signals(
    spot_prices,
    forward_prices,
    volatility
)

# Extend to all assets (metals get 0 carry)
carry_signals = carry_signals.reindex(columns=all_prices.columns).fillna(0)
print(f"  Shape: {carry_signals.shape}")

In [ ]:
# Generate regime multiplier
print("Generating regime multiplier...")
if 'VIX' in vol_data.columns:
    _, regime_multiplier = regime_engine.calculate_regime_series(vol_data['VIX'])
    regime_multiplier = regime_multiplier.reindex(all_prices.index).fillna(1.0)
else:
    regime_multiplier = pd.Series(1.0, index=all_prices.index)

print(f"  Shape: {regime_multiplier.shape}")

## 4. Construct Portfolio

In [ ]:
# Construct historical portfolio weights
print("Constructing portfolio weights...")
weights = portfolio.construct_portfolio(
    momentum_signals=momentum_signals,
    carry_signals=carry_signals,
    returns=returns,
    regime_multiplier=regime_multiplier,
)

print(f"\nWeights shape: {weights.shape}")
print(f"Date range: {weights.index[0]} to {weights.index[-1]}")

In [ ]:
# Check weight distribution
print("\nWeight Statistics:")
print(weights.describe().round(3))

## 5. Run Backtest

In [ ]:
# Run backtest
print("Running backtest...")
result = backtest_engine.run_backtest(
    weights=weights,
    returns=returns,
)

In [ ]:
# Display key metrics
metrics = result.metrics

print("="*60)
print("BACKTEST RESULTS")
print("="*60)
print(f"\nPerformance Metrics:")
print(f"  Sharpe Ratio:      {metrics['sharpe_ratio']:.2f}")
print(f"  Annual Return:     {metrics['ann_return']*100:+.1f}%")
print(f"  Annual Volatility: {metrics['ann_volatility']*100:.1f}%")
print(f"  Max Drawdown:      {metrics['max_drawdown']*100:.1f}%")
print(f"\nRisk-Adjusted:")
print(f"  Sortino Ratio:     {metrics['sortino_ratio']:.2f}")
print(f"  Calmar Ratio:      {metrics['calmar_ratio']:.2f}")
print(f"\nTrading:")
print(f"  Win Rate:          {metrics['win_rate']*100:.0f}%")
print(f"  Profit Factor:     {metrics['profit_factor']:.2f}")
print(f"  Periods:           {metrics['num_periods']}")
print("="*60)

## 6. Performance Visualization

In [ ]:
# Create performance charts
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Cumulative returns
ax1 = axes[0, 0]
result.cumulative_returns.plot(ax=ax1, color='blue', linewidth=1.5)
ax1.set_title('Cumulative Returns', fontsize=12)
ax1.set_xlabel('')
ax1.set_ylabel('Growth of $1')
ax1.grid(True, alpha=0.3)

# Drawdowns
ax2 = axes[0, 1]
(result.drawdowns * 100).plot(ax=ax2, color='red', linewidth=1)
ax2.fill_between(result.drawdowns.index, result.drawdowns * 100, 0, color='red', alpha=0.3)
ax2.set_title('Drawdowns', fontsize=12)
ax2.set_xlabel('')
ax2.set_ylabel('Drawdown %')
ax2.grid(True, alpha=0.3)

# Monthly returns heatmap
ax3 = axes[1, 0]
monthly_returns = result.returns.resample('M').apply(lambda x: (1+x).prod()-1)
monthly_pct = monthly_returns * 100
ax3.hist(monthly_pct, bins=30, color='steelblue', edgecolor='white')
ax3.axvline(x=0, color='black', linestyle='-', linewidth=1)
ax3.axvline(x=monthly_pct.mean(), color='red', linestyle='--', label=f'Mean: {monthly_pct.mean():.1f}%')
ax3.set_title('Monthly Returns Distribution', fontsize=12)
ax3.set_xlabel('Monthly Return %')
ax3.set_ylabel('Frequency')
ax3.legend()

# Rolling Sharpe
ax4 = axes[1, 1]
rolling_metrics = backtest_engine.calculate_rolling_metrics(result.returns, window=252)
if 'sharpe' in rolling_metrics.columns:
    rolling_metrics['sharpe'].plot(ax=ax4, color='green', linewidth=1)
    ax4.axhline(y=0.5, color='orange', linestyle='--', alpha=0.7, label='Target (0.5)')
    ax4.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax4.set_title('Rolling 1-Year Sharpe Ratio', fontsize=12)
    ax4.set_xlabel('')
    ax4.set_ylabel('Sharpe Ratio')
    ax4.legend()
    ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Attribution Analysis

In [ ]:
# Return attribution by asset
attribution = backtest_engine.attribution_analysis(weights, returns)

print("\nReturn Attribution by Asset:")
display(attribution.round(4))

In [ ]:
# Plot attribution
fig, ax = plt.subplots(figsize=(10, 5))

colors = ['green' if c > 0 else 'red' for c in attribution['contribution']]
ax.barh(attribution.index, attribution['contribution'] * 100, color=colors)
ax.set_xlabel('Contribution to Total Return (%)')
ax.set_title('Return Attribution by Asset')
ax.axvline(x=0, color='black', linewidth=0.5)
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 8. Regime Analysis

In [ ]:
# Analyze performance by regime
if 'VIX' in vol_data.columns:
    vix_aligned = vol_data['VIX'].reindex(result.returns.index).fillna(method='ffill')
    regimes = vix_aligned.apply(regime_engine.detect_vix_regime)
    
    regime_performance = {}
    for regime in ['low', 'normal', 'elevated', 'high', 'extreme']:
        mask = regimes == regime
        if mask.sum() > 0:
            regime_returns = result.returns[mask]
            regime_performance[regime] = {
                'count': mask.sum(),
                'pct_time': mask.sum() / len(mask) * 100,
                'avg_return': regime_returns.mean() * 252 * 100,
                'volatility': regime_returns.std() * np.sqrt(252) * 100,
                'sharpe': (regime_returns.mean() * 252) / (regime_returns.std() * np.sqrt(252)) if regime_returns.std() > 0 else 0,
            }
    
    regime_df = pd.DataFrame(regime_performance).T
    print("\nPerformance by VIX Regime:")
    display(regime_df.round(2))

## 9. Summary

In [ ]:
# Final summary
target_sharpe = 0.55
achieved_sharpe = metrics['sharpe_ratio']
sharpe_delta = achieved_sharpe - target_sharpe

print("="*60)
print("BACKTEST SUMMARY")
print("="*60)
print(f"\nTarget Sharpe:   {target_sharpe:.2f}")
print(f"Achieved Sharpe: {achieved_sharpe:.2f}")
print(f"Delta:           {sharpe_delta:+.2f}")

if achieved_sharpe >= target_sharpe:
    print("\n✓ TARGET ACHIEVED!")
else:
    print(f"\n✗ Target not met (shortfall: {abs(sharpe_delta):.2f})")

print(f"\nKey Findings:")
print(f"  - Best performing asset: {attribution['contribution'].idxmax()}")
print(f"  - Worst performing asset: {attribution['contribution'].idxmin()}")
print(f"  - Average regime multiplier: {regime_multiplier.mean():.2f}")
print("="*60)

In [ ]:
# Cache stats
print(f"\nCache Statistics:")
print(f"  Entries: {cache.size}")
print(f"  Hit Rate: {cache.stats['hit_rate']:.1%}")

---

**End of Backtest Report**